# Computer Assignment 1
## Ali Bahari

Project Name: Computer Assignment 1

Project Goal: Use informed and uninformed search algorithms to solve the problem of this assignment which starting form a point, delivering some balls to theirs destinations and finally exiting the maze

In this assignment we will start from an initial point in a maze. We can go up, down, left or right in the maze unless there is a barrier in the way. There are some balls in this maze at different locations the goal is to go to these locations, pick up the balls and finally deliver them to their destination and the thing is we have a certain capacity and we can't pick up more balls then the capacity. After delivering all the balls we have to navigate to the end point and exit the maze. Using informed and uninformed search algorithms such as BFS, IDS, A star and weighted A star, we will be able to achieve our goal.



Everything has been implemented, tested and run on Linux but the report has been done on Windows!

# Modeling:
(There will be more detailed explanation further down the report when State class is declared!)

Initial State: Maze read from the provided files all the balls are in place none of them is delivered and we are at start point

Goal State: Maze that every ball has been delivered to its destination and we are at the exit of the maze

Actions: Up, down, left, right, picking the ball

Transition model: Moving and picking up balls will result in changes in the State parameters

Path cost: Steps including moving and picking up balls we have taken so far

Optimal solution: Final answer and the path we have taken to get to goal state which is the result of the codes below

# State Class:

Each state is defined by these parameters:

curLoc_x and curLoc_y => current location of us in the maze

curCapacity => current capacity of our bag

curNumRemainingBalls => number of balls that haven't been delieverd and are still remaining

needDelivery => Destination location of the balls that we have picked and we have to deliver them

hasPicked => Initial location of the balls that we have picked up (It doesn't matter we have delivered them or not if we pick some ball up its location will be stored here)

parentState => Previous state that we were in before being in current state

depth => Length of the path that we have taken to get to this current state (Remember picking up a ball counts as a move)

parentState is used for printing the path at the end

depth is used in A star and weighted A star for calculating g(n)

### getHash:
This function gives every state a unique name and identity. If we want to identify a state we will use this method.

For optimization we want to have states as few as possible meaning that we don't want to be extremely specific when naming a state but also we want to incorporate as many parameters as possible in naming process. So after checking different possibilities I came up with the hashing process shown below.

For hashing we need parameters like: location of us, number of remaining balls, destionations for delivery and finally the parent state. For parent state only the location is used and for the start state parent is considered -00

Hashing process (giving the state a specific identity) is used in search algorithms which will be explained further down the report

Also in BFS we can optimize even more and use getHashBFS to make the number of states even less. For faster BFS runtime getHashBFS can be used since the number of states will be lower than getHash. Both getHash and getHashBFS were used and the results are shown further down the report for each these methods. All in all getHash was used for all of the algorithms but for BFS getHashBFS was also used to get the results even faster.

In [1]:
class State():
    def __init__(self, curLoc_x , curLoc_y , curCapacity , curNumRemainingBalls , needDelivery , hasPicked ,
                 parentState=None , depth=0):

        self.curLoc_x = curLoc_x
        self.curLoc_y = curLoc_y
        self.curCapacity = curCapacity
        self.curNumRemainingBalls = curNumRemainingBalls
        self.needDelivery = needDelivery
        self.parentState = parentState
        self.hasPicked = hasPicked

        if(self.parentState is None):
            self.depth = 0
        else:
            self.depth = self.parentState.depth + 1

#     def __eq__(self , other):
#         return self.curLoc_x == other.curLoc_x and self.curLoc_y == other.curLoc_y and \
#                self.curNumRemainingBalls == other.curNumRemainingBalls

    def getHash(self):
        strNeedDelivery = "".join([str(t) for t in self.needDelivery])
        if(self.parentState is None):
            strParent = "-00"
        else:
            strParent = str(self.parentState.curLoc_x) + str(self.parentState.curLoc_y)

        return str(self.curLoc_x) + str(self.curLoc_y) + str(self.curNumRemainingBalls) + strNeedDelivery + strParent

    def getHashBFS(self):
        strNeedDelivery = "".join([str(t) for t in self.needDelivery])
        return str(self.curLoc_x) + str(self.curLoc_y) + str(self.curNumRemainingBalls) + strNeedDelivery

    def getNumPicked(self):
        return len(self.hasPicked)

    def getCurLoc_x(self):
        return self.curLoc_x

    def getCurLoc_y(self):
        return self.curLoc_y

    def getCurCapacity(self):
        return self.curCapacity

    def getCurNumRemainingBalls(self):
        return self.curNumRemainingBalls

    def getNeedDelivery(self):
        return self.needDelivery

    def getHasPicked(self):
        return self.hasPicked

    def getParentState(self):
        return self.parentState

    def getDepth(self):
        return self.depth

    def setCurCapacity(self , newValue):
        self.curCapacity = newValue

    def setCurNumRemainingBalls(self , newValue):
        self.curNumRemainingBalls = newValue

    def setNeedDelivery(self , newValue):
        self.needDelivery = newValue

    def setHasPicked(self , newValue):
        self.hasPicked = newValue

Before starting let's first implement some tools that will be useful in the codes

In [2]:
def getMazeElement(maze , x , y):
    return maze[x][y * 2]


def isValidInMaze(maze , x , y , num_rows , num_columns):
    if(x >= 0 and y >= 0 and x < num_rows and y < num_columns and getMazeElement(maze , x , y) != '*'):
        return True

    return False

getMazeElement is used for getting the element that is in a specific location of the maze.

isValidInMaze is for checking if we are in the maze and that we are not on a barrier

# BFS:

The algorithm is the same as the one explained in the course lectures.

Frontier is implemented as a list which behaves like a queue => enqueue: append , dequeue: pop(0)

Explored is implemented as a set. Since we will search explored for already existing states we need the search to be as fast as possible. Set() is a good options for this situation. getHash() method that I explained above will be used here for searching and adding to set.

For any new state we make, there's another node added node to frontier. Also the node's hash which is received from getHash() method is added to explored set. In BFS we can be sure that if we see a new state which was added into the frontier or explored set before, it doesn't have to be added to the frontier again. This assumption is not correct for A star algorithm with will be explained in A star part of the report.

In [3]:
def updateFrontier(frontier , newState , goal_x , goal_y , explored):
    frontier.append(newState)
    explored.add(newState.getHashBFS())

    if(newState.getCurLoc_x() == goal_x and newState.getCurLoc_y() == goal_y and newState.getCurNumRemainingBalls() == 0):
        return True

    return False

updateFrontier => add new state to frontier queue, add getHash() value of the new state to explored and finally check if we are done

In [4]:
def placeBall(maze , curState , ballsDestLocs):
    if(len(curState.getNeedDelivery()) <= 0):
        return

    try:
        deliveryIndex = curState.getNeedDelivery().index((curState.getCurLoc_x() , curState.getCurLoc_y()))
    except ValueError:
        return

    curStateNeedDelivery = list(curState.getNeedDelivery())
    del curStateNeedDelivery[deliveryIndex]
    curState.setNeedDelivery(curStateNeedDelivery)

    curState.setCurCapacity(curState.getCurCapacity() + 1)
    curState.setCurNumRemainingBalls(curState.getCurNumRemainingBalls() - 1)

placeBall => check if we are on a delivery destination and if yes deliver the ball

In [5]:
def grabBall(maze , curState , ballsInitLocs , ballsDestLocs , frontier , explored):
    if(curState.getCurCapacity() <= 0):
        return

    try:
        ballIndex = ballsInitLocs.index((curState.getCurLoc_x() , curState.getCurLoc_y()))
    except ValueError:
        return

    if((curState.getCurLoc_x() , curState.getCurLoc_y()) in curState.getHasPicked()):
        return

    newNeedDelivery = list(curState.getNeedDelivery())
    newNeedDelivery.append(ballsDestLocs[ballIndex])

    newHasPicked = list(curState.getHasPicked())
    newHasPicked.append((curState.getCurLoc_x() , curState.getCurLoc_y()))

    newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() , curState.getCurCapacity() - 1 , 
                     curState.getCurNumRemainingBalls() , newNeedDelivery , newHasPicked , curState)

    if(newState.getHashBFS() not in explored):
        frontier.append(newState)
        explored.add(newState.getHashBFS())

grabBall => check if we are on a never seen ball initial location and if yes grab the ball

grabbing the ball is counted as a move therefore it'll create a new state so we need to add the new state to frontier and also add the hash of it to explored

In [6]:
def BFS(maze , num_rows , num_columns , start_x , start_y , goal_x , goal_y , capacity , num_balls , ballsInitLocs ,
        ballsDestLocs):

    frontier = []
    explored = set()

    start_state = State(start_x , start_y , capacity , num_balls , [] , [])
    frontier.append(start_state)
    explored.add(start_state.getHashBFS())

    uniqueExplored = set()
    numExploredStates = 0; numUniqueExploredStates = 0

    if(start_x == goal_x and start_y == goal_y and num_balls == 0):
        return frontier[0] , numExploredStates , numUniqueExploredStates

    while(len(frontier) != 0):
        numExploredStates += 1
        curState = frontier.pop(0)

        if(curState.getHashBFS() not in uniqueExplored):
            numUniqueExploredStates += 1
            uniqueExplored.add(curState.getHashBFS())

        grabBall(maze , curState , ballsInitLocs , ballsDestLocs , frontier , explored)
        placeBall(maze , curState , ballsDestLocs)

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , num_rows , num_columns)): # Left
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHashBFS() not in explored):
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Down
            newState = State(curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHashBFS() not in explored):
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Up
            newState = State(curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHashBFS() not in explored):
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , num_rows , num_columns)): # Right
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHashBFS() not in explored):
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored)):
                    return newState , numExploredStates , numUniqueExploredStates

In BFS each time we'll get the first state in frontier queue then we will check different possible actions for example if it's possible to place or grab ball it would be done an after that we will check the up, down, left and right for next states and if everything was ok after the checking, explored and frontier will be updated. We keep doing these processes until the queue is empty or we have reached the goal state.


Counting the number of explored states:

Every state that is popped from queue is counted as an explored state

Counting the number of unique explored states:

uniqueExplored set is declered in which every popped state's hash is added and if we want to increment number of unique explored states we have to check if we have already explored it before or not by searching in this set. By having this condition the counted explored states will be unique.

### Getting Input:

Now After implementation it's time to get the input values from provided files and run BFS.

In [7]:
def getNumAndPrintActions(firstState , printActions = False):
    num_levels = 0
    while(firstState.getParentState() is not None):
        if(printActions):
            print(str(firstState.getCurLoc_x()) + " " + str(firstState.getCurLoc_y()) + 
                    " capacity: " + str(firstState.getCurCapacity()) )
        firstState = firstState.getParentState()
        num_levels += 1

    return num_levels

getNumAndPrintActions will print the path and the length of it as the final result. Printing the path is and optional part of this function by passing value True as the second parameter of this function the path will be printed out from goal to start state.

Test case 1 is used here as an example.

In [8]:
import time

testFile = open("Tests/1.txt" , "r")
inputs = testFile.read().splitlines()
testFile.close()

num_rows , num_columns = list(map(int , inputs[0].split()))
start_x , start_y = list(map(int , inputs[1].split()))
goal_x , goal_y = list(map(int , inputs[2].split()))
capacity = int(inputs[3])
num_balls = int(inputs[4])
ballsInitLocs = []
ballsDestLocs = []
for i in range(num_balls):
    s_x , s_y , d_x , d_y = list(map(int , inputs[5 + i].split()))
    ballsInitLocs.append((s_x , s_y)); ballsDestLocs.append((d_x , d_y))

maze = inputs[5 + num_balls :]


tic = time.time()
goalState , numExploredStates , numUniqueExploredStates = BFS(maze , num_rows , num_columns , start_x , start_y ,
                                                              goal_x , goal_y , capacity , num_balls , ballsInitLocs ,
                                                              ballsDestLocs)
toc = time.time()


num_levels = getNumAndPrintActions(goalState)

print("Num of levels: " + str(num_levels))
print("Number of explored States: " + str(numExploredStates))
print("Number of unique explored States: " + str(numUniqueExploredStates))
print("Time: " + str(toc - tic) + " s")

Num of levels: 20
Number of explored States: 125
Number of unique explored States: 125
Time: 0.004502058029174805 s


# IDS:

The Algorithm is similar to the one explained in the course lectures.

There will be a maxDepth that we will do dfs until we get to that depth. Other parts will be explained below

In [9]:
def grabBall(maze , curState , ballsInitLocs , ballsDestLocs):

    if(curState.getCurCapacity() <= 0):
        return None

    try:
        ballIndex = ballsInitLocs.index((curState.getCurLoc_x() , curState.getCurLoc_y()))
    except ValueError:
        return None

    if((curState.getCurLoc_x() , curState.getCurLoc_y()) in curState.getHasPicked()):
        return None

    newNeedDelivery = list(curState.getNeedDelivery())
    newNeedDelivery.append(ballsDestLocs[ballIndex])

    newHasPicked = list(curState.getHasPicked())
    newHasPicked.append((curState.getCurLoc_x() , curState.getCurLoc_y()))

    return State(curState.getCurLoc_x() , curState.getCurLoc_y() , curState.getCurCapacity() - 1 , 
                 curState.getCurNumRemainingBalls() , newNeedDelivery , newHasPicked , curState)

placeBall is exactly like BFS but grabBall has been changed and instead of adding to explored or frontier we will return the new state.

In [10]:
def DFS(curState , maxDepth , currentPathLength , allExplored , allExploredBestLengths , dfsExplored , dfsExploredLengths ,
        maze , num_rows , num_columns , numExploredStates , numUniqueExploredStates):

    numExploredStates += 1

    if(curState.getCurLoc_x() == goal_x and curState.getCurLoc_y() == goal_y and curState.getCurNumRemainingBalls() == 0):
        return True , curState , numExploredStates , numUniqueExploredStates

    if(maxDepth <= currentPathLength):
        return False , None , numExploredStates , numUniqueExploredStates

    # Because of starting state the second condition must be > instead of >=
    if ( (curState.getHash() in dfsExplored and currentPathLength >= dfsExploredLengths[curState.getHash()]) or 
         (curState.getHash() in allExplored and currentPathLength > allExploredBestLengths[curState.getHash()]) ):

        return False , None , numExploredStates , numUniqueExploredStates

    if(curState.getHash() not in dfsExplored and curState.getHash() not in allExplored):
        numUniqueExploredStates += 1

    allExplored.add(curState.getHash()); allExploredBestLengths[curState.getHash()] = currentPathLength
    dfsExplored.add(curState.getHash()); dfsExploredLengths[curState.getHash()] = currentPathLength

    placeBall(maze , curState , ballsDestLocs)
    possibleNewState = grabBall(maze , curState , ballsInitLocs , ballsDestLocs)

    if(possibleNewState is not None):
        result , resultState , numExploredStates , numUniqueExploredStates = DFS(possibleNewState , maxDepth ,
                                                                                 currentPathLength , allExplored ,
                                                                                 allExploredBestLengths , dfsExplored ,
                                                                                 dfsExploredLengths , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates);
        if(result):
            return result , resultState , numExploredStates , numUniqueExploredStates


    if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , num_rows , num_columns)): # Left
        newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , curState.getCurCapacity() ,
                         curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                         curState)

        result , resultState , numExploredStates , numUniqueExploredStates = DFS(newState , maxDepth ,
                                                                                 currentPathLength + 1 , allExplored ,
                                                                                 allExploredBestLengths , dfsExplored ,
                                                                                 dfsExploredLengths , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates)
        if(result):
            return result , resultState , numExploredStates , numUniqueExploredStates

    if(isValidInMaze(maze , curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Down
        newState = State(curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                         curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                         curState)

        result , resultState , numExploredStates , numUniqueExploredStates = DFS(newState , maxDepth ,
                                                                                 currentPathLength + 1 , allExplored ,
                                                                                 allExploredBestLengths , dfsExplored ,
                                                                                 dfsExploredLengths , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates)
        if(result):
            return result , resultState , numExploredStates , numUniqueExploredStates

    if(isValidInMaze(maze , curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Up
        newState = State(curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                         curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                         curState)

        result , resultState , numExploredStates , numUniqueExploredStates = DFS(newState , maxDepth ,
                                                                                 currentPathLength + 1 , allExplored ,
                                                                                 allExploredBestLengths , dfsExplored ,
                                                                                 dfsExploredLengths , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates)
        if(result):
            return result , resultState , numExploredStates , numUniqueExploredStates

    if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , num_rows , num_columns)): # Right
        newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , curState.getCurCapacity() ,
                         curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                         curState)

        result , resultState , numExploredStates , numUniqueExploredStates = DFS(newState , maxDepth ,
                                                                                 currentPathLength + 1 , allExplored ,
                                                                                 allExploredBestLengths , dfsExplored ,
                                                                                 dfsExploredLengths , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates)
        if(result):
            return result , resultState , numExploredStates , numUniqueExploredStates


    return False , None , numExploredStates , numUniqueExploredStates

For optimization 4 sets are declared dfsExplored , allExplored , dfsExploredLengths , allExploredBestLengths

These sets are used for stoping the exploration of states that are not useful meaning we don't explore the states that are explored in the current DFS run with more or equal depth value than previously stored for that particular state and we also don't explore the states that were explored in all previous DFS runs with more depth value than previously stored for that particular state. This process will reduce explored states by a lot and optimize our approach for the better.

After the process above these sets will be updated and just like BFS for every action we will have a new state so after checking the validity of the action we'll call DFS for each of them.

For numExploredStates every DFS run will increment it and for numUniqueExploredStates we can use dfsExplored and allExplored to check the uniqueness of the states and increment numUniqueExploredStates accordingly.

result is boolean value for checking whether we have reached goal state or not and resultState is visited state before returning.

In [11]:
def IDS(maze , num_rows , num_columns , start_x , start_y , goal_x , goal_y , capacity , num_balls , ballsInitLocs ,
        ballsDestLocs):

    idsMaxDepth = 1
    allExplored = set(); allExploredBestLengths = {}
    result = False
    numExploredStates = 0; numUniqueExploredStates = 0

    start_state = State(start_x , start_y , capacity , num_balls , [] , [])

    while(not result):
        result , resultState , numExploredStates , numUniqueExploredStates = DFS(start_state , idsMaxDepth ,
                                                                                 0 , allExplored ,
                                                                                 allExploredBestLengths , set() ,
                                                                                 {} , maze , num_rows ,
                                                                                 num_columns , numExploredStates ,
                                                                                 numUniqueExploredStates)
        idsMaxDepth += 1

    return resultState , numExploredStates , numUniqueExploredStates

IDS will run DFS each time with different idsMaxDepth until finally it reaches the goal state.

Getting input from file and running the implementation is exactly similar to BFS but here we will run IDS instead.

Test case 1 is used here as an example.

In [12]:
testFile = open("Tests/1.txt" , "r")
inputs = testFile.read().splitlines()
testFile.close()

num_rows , num_columns = list(map(int , inputs[0].split()))
start_x , start_y = list(map(int , inputs[1].split()))
goal_x , goal_y = list(map(int , inputs[2].split()))
capacity = int(inputs[3])
num_balls = int(inputs[4])
ballsInitLocs = []
ballsDestLocs = []
for i in range(num_balls):
    s_x , s_y , d_x , d_y = list(map(int , inputs[5 + i].split()))
    ballsInitLocs.append((s_x , s_y)); ballsDestLocs.append((d_x , d_y))

maze = inputs[5 + num_balls :]


tic = time.time()
goalState , numExploredStates , numUniqueExploredStates = IDS(maze , num_rows , num_columns , start_x , start_y ,
                                                              goal_x , goal_y , capacity , num_balls , ballsInitLocs ,
                                                              ballsDestLocs)
toc = time.time()


num_levels = getNumAndPrintActions(goalState)

print("Num of levels: " + str(num_levels))
print("Number of explored States: " + str(numExploredStates))
print("Number of unique explored States: " + str(numUniqueExploredStates))
print("Time: " + str(toc - tic) + " s")

Num of levels: 20
Number of explored States: 6991
Number of unique explored States: 361
Time: 0.09694051742553711 s


# A star:

The Algorithm is similar to the one explained in the course lectures but there are differences in implementation.

Expanding states is done based on f(n) and the one with the lowest f(n) will get expanded. This situation can be implemented in a really elegent way using minimum heap. So first of let's implement a Min Heap class.

In [13]:
from heapq import heapify, heappush, heappop

class MinHeap():
    def __init__(self):
        self.heap = []
        heapify(self.heap)
        self.index = 0


    def push(self , element , priority):
        heappush(self.heap , (priority , self.index , element))
        self.index += 1 # for elements with equal priority

    def pop(self):
        return heappop(self.heap)[-1]

    def top(self):
        return self.heap[0]

    def isEmpty(self):
        if(len(self.heap) == 0):
            return True
        return False

    def printHeap(self):
        for priority , _ , element in self.heap:
            print("Priority: " + str(priority) + " element: " + str(element))

In this implementation priority will be f(n) and element will be the state. Also if f(n) of two states are equal we use an index so when chosing these states it'll be something like a queue.

placeBall will be the same like before but this time grabBall and updateFrontier are changed as well.

In [14]:
def updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs):
    frontier.push(newState , calculateF(newState , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs))
    explored.add(newState.getHash())

    if(newState.getCurLoc_x() == goal_x and newState.getCurLoc_y() == goal_y and newState.getCurNumRemainingBalls() == 0):
        return True

    return False

Since frontier is implemented as a min heap we have to push it in the heap with its f(n) value. Except this other things are the same as before.

calculateF is implemented further down the report.

In [15]:
def grabBall(maze , curState , ballsInitLocs , ballsDestLocs , frontier , goal_x , goal_y , num_balls , explored ,
             numExploredStates):

    if(curState.getCurCapacity() <= 0):
        return numExploredStates

    try:
        ballIndex = ballsInitLocs.index((curState.getCurLoc_x() , curState.getCurLoc_y()))
    except ValueError:
        return numExploredStates

    if((curState.getCurLoc_x() , curState.getCurLoc_y()) in curState.getHasPicked()):
        return numExploredStates

    newNeedDelivery = list(curState.getNeedDelivery())
    newNeedDelivery.append(ballsDestLocs[ballIndex])

    newHasPicked = list(curState.getHasPicked())
    newHasPicked.append((curState.getCurLoc_x() , curState.getCurLoc_y()))

    newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() , curState.getCurCapacity() - 1 , 
                     curState.getCurNumRemainingBalls() , newNeedDelivery , newHasPicked , curState)

    if(newState.getHash() not in explored):
        frontier.push(newState , calculateF(newState , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs))
        explored.add(newState.getHash())
        return numExploredStates + 1

    return numExploredStates

Implementing frontier as a min heap has the same effect on the code as updateFrontier but here numExploredStates is also changed since grabbing the ball counts as a new state.

## Heuristics:

#### First heuristic: 

The fisrt heuristic takes multiple different situations into account.

First of all it checks whether we have remaining balls or not if we don't have any left so this means we have to go to the exit so the best exploration is the one that leads to the exit location. Manhattan distance from new location to exit location is a good option.

But what if we had remaining balls in this situation?

Then we'll take a look at our deliveries left! if we had no deliveries left so this means we have to go at least get a ball and then think about doing something else from that point forward. Therefore we have to find the nearest ball to us furthermore the distance from that ball to its destination shouldn't be too long, as a result we have to take this parameter into account as well, based on all of this info the manhattan distance from new location to the nearest ball with the lowest distance to its delivery point looks like a good option.

But again what if we had some deliveries?

Then we have to look at our bag! Is there any capacity? if no we have to deliver at least one ball in order to continue so in our deliveries we have to find the nearest delivery point and deliver that ball. Manhattan distance from new location to the mentioned location is the best option. But if yes then we have two options either go deliver a ball or no go grab a ball in this case minimum manhattan distance of these two options seems like a good approach.

This heuristic is admissible since we'll always use minimum possible value.

#### Second heuristic: 

The second heuristic takes a few situations into account.

First part is exactly like first heuristic for checking the number of remaining balls. If no more balls left manhattan distance from new location to the exit location is considered but if there is any ball left it only takes the number of remaining balls into account.

This heuristic is consistent since no matter what states are chosen the following equation will stand:

cost(A to B) >= h(A) - h(B)

Since the heuristic is consistent then it'll be admissible as well.

#### f(n):

The path cost is the distance that we have gone which is the depth field of state class plus the number of balls we have picked since picking up balls counts as a move.

Sum of path cost and heuristic result value will be f(n).

In [16]:
def findManhattanDistance(x1 , y1 , x2 , y2):
    return abs(x2 - x1) + abs(y2 - y1)


def findMinHeuPicking(curState , ballsInitLocs , ballsDestLocs , num_balls , hasPicked):
    minCost = float("inf")

    for i in range(num_balls):
        # plus 1 for picking the ball up
        if(ballsInitLocs[i] not in hasPicked):
            cost = findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , ballsInitLocs[i][0] , ballsInitLocs[i][1]) + \
                   findManhattanDistance(ballsInitLocs[i][0] , ballsInitLocs[i][1] , ballsDestLocs[i][0] , ballsDestLocs[i][1]) + 1

            if(cost < minCost):
                minCost = cost

    return minCost


def findMinHeuDelivery(curState):
    minCost = float("inf")

    for put_x , put_y in curState.getNeedDelivery():
        cost = findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , put_x , put_y)

        if(cost < minCost):
            minCost = cost

    return minCost


def calculateF(curState , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs):
    # Picking up a ball is a move so it's used in f calculation

    # Heuristic 1 --> Surely Admissible
    if(curState.getCurNumRemainingBalls() == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , goal_x , goal_y)

    if(len(curState.getNeedDelivery()) == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               findMinHeuPicking(curState , ballsInitLocs , ballsDestLocs , num_balls , curState.getHasPicked())

    if(curState.getCurCapacity() == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               findMinHeuDelivery(curState)

    return curState.getDepth() + curState.getNumPicked() + \
           min(findMinHeuDelivery(curState) ,
               findMinHeuPicking(curState , ballsInitLocs , ballsDestLocs , num_balls , curState.getHasPicked()))

#     # Heuristic 2 --> Surely Consistent , Addmissible
#     if(curState.getCurNumRemainingBalls() == 0):
#         return curState.getDepth() + curState.getNumPicked() + \
#                 findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , goal_x , goal_y)

#     return curState.getDepth() + curState.getNumPicked() + curState.getCurNumRemainingBalls()

Other parts are similar to BFS implementation except we have to remember that frontier is a min heap instead of a queue.

In [17]:
def A_Star(maze , num_rows , num_columns , start_x , start_y , goal_x , goal_y , capacity , num_balls , ballsInitLocs ,
           ballsDestLocs):

    frontier = MinHeap()
    explored = set()

    start_state = State(start_x , start_y , capacity , num_balls , [] , [])
    frontier.push(start_state , calculateF(start_state , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs))
    explored.add(start_state.getHash())

    uniqueExplored = set()
    numExploredStates = 0; numUniqueExploredStates = 0

    if(start_x == goal_x and start_y == goal_y and num_balls == 0):
        return frontier.top() , numExploredStates , numUniqueExploredStates

    while(not frontier.isEmpty()):
        curState = frontier.pop()

        if(curState.getHash() not in uniqueExplored):
            numUniqueExploredStates += 1
            uniqueExplored.add(curState.getHash())

        numExploredStates = grabBall(maze , curState , ballsInitLocs , ballsDestLocs , frontier , goal_x , goal_y ,
                                     num_balls , explored , numExploredStates)
        placeBall(maze , curState , ballsDestLocs)

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , num_rows , num_columns)): # Left
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Down
            newState = State(curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Up
            newState = State(curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , num_rows , num_columns)): # Right
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

Getting input from file and running the implementation is exactly similar to previous parts but here we will run A star instead.

Test case 1 is used here as an example.

In [18]:
testFile = open("Tests/1.txt" , "r")
inputs = testFile.read().splitlines()
testFile.close()

num_rows , num_columns = list(map(int , inputs[0].split()))
start_x , start_y = list(map(int , inputs[1].split()))
goal_x , goal_y = list(map(int , inputs[2].split()))
capacity = int(inputs[3])
num_balls = int(inputs[4])
ballsInitLocs = []
ballsDestLocs = []
for i in range(num_balls):
    s_x , s_y , d_x , d_y = list(map(int , inputs[5 + i].split()))
    ballsInitLocs.append((s_x , s_y)); ballsDestLocs.append((d_x , d_y))

maze = inputs[5 + num_balls :]


tic = time.time()
goalState , numExploredStates , numUniqueExploredStates = A_Star(maze , num_rows , num_columns , start_x , start_y ,
                                                                 goal_x , goal_y , capacity , num_balls ,
                                                                 ballsInitLocs , ballsDestLocs)
toc = time.time()


num_levels = getNumAndPrintActions(goalState)

print("Num of levels: " + str(num_levels))
print("Number of explored States: " + str(numExploredStates))
print("Number of unique explored States: " + str(numUniqueExploredStates))
print("Time: " + str(toc - tic) + " s")

Num of levels: 20
Number of explored States: 330
Number of unique explored States: 294
Time: 0.015485763549804688 s


# Weighted A star:

Everything here is exactly similar to A star implementation above but there will be a difference in calculateF function.

In [19]:
def calculateF(curState , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs):
    # Picking up a ball is a move so it's used in f calculation

    # # First Alfa
    # alfa = 1.5

    # Second Alfa
    alfa = 5

    # Heuristic 1 --> Surely Admissible
    if(curState.getCurNumRemainingBalls() == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               alfa * (findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , goal_x , goal_y))

    if(len(curState.getNeedDelivery()) == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               alfa * (findMinHeuPicking(curState , ballsInitLocs , ballsDestLocs , num_balls , curState.getHasPicked()))

    if(curState.getCurCapacity() == 0):
        return curState.getDepth() + curState.getNumPicked() + \
               alfa * (findMinHeuDelivery(curState))

    return curState.getDepth() + curState.getNumPicked() + \
           alfa * (min(findMinHeuDelivery(curState) ,
                       findMinHeuPicking(curState , ballsInitLocs , ballsDestLocs , num_balls , curState.getHasPicked())))

#     # Heuristic 2 --> Surely Consistent , Addmissible
#     if(curState.getCurNumRemainingBalls() == 0):
#         return curState.getDepth() + curState.getNumPicked() + \
#                 alfa * (findManhattanDistance(curState.getCurLoc_x() , curState.getCurLoc_y() , goal_x , goal_y))

#     return curState.getDepth() + curState.getNumPicked() + alfa * (curState.getCurNumRemainingBalls())

In calculating f(n) for weighted A star we have to multiply heuristic value by an alfa. Here 1.5 and 5 have been chosen for alfa value. So the path cost will stay the same but h(n) value must be multiplied by 1.5 or 5. Everything else is exactly the same.

In [20]:
def weighted_A_Star(maze , num_rows , num_columns , start_x , start_y , goal_x , goal_y , capacity , num_balls , ballsInitLocs , ballsDestLocs):
    frontier = MinHeap()
    explored = set()

    start_state = State(start_x , start_y , capacity , num_balls , [] , [])
    frontier.push(start_state , calculateF(start_state , goal_x , goal_y , num_balls , ballsInitLocs , ballsDestLocs))
    explored.add(start_state.getHash())

    uniqueExplored = set()
    numExploredStates = 0; numUniqueExploredStates = 0

    if(start_x == goal_x and start_y == goal_y and num_balls == 0):
        return frontier.top() , numExploredStates , numUniqueExploredStates

    while(not frontier.isEmpty()):
        curState = frontier.pop()

        if(curState.getHash() not in uniqueExplored):
            numUniqueExploredStates += 1
            uniqueExplored.add(curState.getHash())

        numExploredStates = grabBall(maze , curState , ballsInitLocs , ballsDestLocs , frontier , goal_x , goal_y , num_balls , explored , numExploredStates)
        placeBall(maze , curState , ballsDestLocs)

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , num_rows , num_columns)): # Left
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() - 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Down
            newState = State(curState.getCurLoc_x() + 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , num_rows , num_columns)): # Up
            newState = State(curState.getCurLoc_x() - 1 , curState.getCurLoc_y() , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

        if(isValidInMaze(maze , curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , num_rows , num_columns)): # Right
            newState = State(curState.getCurLoc_x() , curState.getCurLoc_y() + 1 , curState.getCurCapacity() ,
                             curState.getCurNumRemainingBalls() , curState.getNeedDelivery() , curState.getHasPicked() ,
                             curState)

            if(newState.getHash() not in explored):
                numExploredStates += 1
                if(updateFrontier(frontier , newState , goal_x , goal_y , explored , num_balls , ballsInitLocs , ballsDestLocs)):
                    return newState , numExploredStates , numUniqueExploredStates

weighted_A_Star function will have the same code as A_Star, only here calculateF has been changed.


Getting input from file and running the implementation is exactly similar to previous parts but here we will run weighted A star instead.

Test case 1 is used here as an example.

In [21]:
testFile = open("Tests/1.txt" , "r")
inputs = testFile.read().splitlines()
testFile.close()

num_rows , num_columns = list(map(int , inputs[0].split()))
start_x , start_y = list(map(int , inputs[1].split()))
goal_x , goal_y = list(map(int , inputs[2].split()))
capacity = int(inputs[3])
num_balls = int(inputs[4])
ballsInitLocs = []
ballsDestLocs = []
for i in range(num_balls):
    s_x , s_y , d_x , d_y = list(map(int , inputs[5 + i].split()))
    ballsInitLocs.append((s_x , s_y)); ballsDestLocs.append((d_x , d_y))

maze = inputs[5 + num_balls :]


tic = time.time()
goalState , numExploredStates , numUniqueExploredStates = weighted_A_Star(maze , num_rows , num_columns , start_x , start_y , goal_x , goal_y ,
                                                                          capacity , num_balls , ballsInitLocs , ballsDestLocs)
toc = time.time()


num_levels = getNumAndPrintActions(goalState)

print("Num of levels: " + str(num_levels))
print("Number of explored States: " + str(numExploredStates))
print("Number of unique explored States: " + str(numUniqueExploredStates))
print("Time: " + str(toc - tic) + " s")

Num of levels: 20
Number of explored States: 338
Number of unique explored States: 288
Time: 0.015468120574951172 s


#### BFS:

BFS can be fast in searching but the hold-up is that it's uninformed. Also BFS does not visit a state twice.

#### IDS:

IDS is much slower than the other search algorithms again one reason being that IDS is uninformed but it's main advantage is it's memory. It can operate with just a stack proportional to the depth of one branch.

#### A star:

A star is a very fast search algorithm a good thing about is that A star is an informed search algorithm so this means it uses a heuristic function to search in the rational direction.

#### Weighted A star:

Weighted A star is completely similar to A star but calculating f(n) is different. This difference will result in even faster search algorithm but the disadvantage can be missing the optimal path.

So each algorithm has it's own advantages and disadvantages so we should use them based on our problem properties.

BFS sometimes can have fewer explored states than A star and that's because it doesn't insert a state which is already in the frontier to frontier again, but A star has to do it, but usually it won't have to explore it so it's always faster.

# Results and Tables:

| TEST CASE 1 |  |  |  |  | 
| :-: | :-: | :-: | :-: | :-: |
| Algorithm | Goal Depth | Number of Explored States | Number of Unique Explored States | Time (s)
| BFS | 20 | 361 | 361 | 0.005358617 |
| BFS with getHashBFS | 20 | 125 | 125 | 0.001468023 |
| IDS | 20 | 6991 | 361 | 0.05544138 |
| A star 1st h(n) | 20 | 330 | 294 | 0.005256255 |
| A star 2nd h(n) | 20 | 304 | 293 | 0.004756967 |
| Weighted A star 1st h(n) 1st Alpha | 20 | 314 | 281 | 0.005456413 |
| Weighted A star 1st h(n) 2nd Alpha | 20 | 338 | 288 | 0.005416822 |
| Weighted A star 2nd h(n) 1st Alpha | 20 | 304 | 293 | 0.004765592 |
| Weighted A star 2nd h(n) 2nd Alpha | 20 | 301 | 275 | 0.004613161 |

| TEST CASE 2 |  |  |  |  | 
| :-: | :-: | :-: | :-: | :-: |
| Algorithm | Goal Depth | Number of Explored States | Number of Unique Explored States | Time (s)
| BFS | 48 | 8756 | 8756 | 0.133912961 |
| BFS with getHashBFS | 48 | 2162 | 2162 | 0.028096676 |
| IDS | 48 | 802915 | 8756 | 5.031510433 |
| A star 1st h(n) | 48 | 2445 | 1196 | 0.024705013 |
| A star 2nd h(n) | 48 | 2445 | 1196 | 0.024365242 |
| Weighted A star 1st h(n) 1st Alpha | 48 | 168 | 48 | 0.001197781 |
| Weighted A star 1st h(n) 2nd Alpha | 48 | 168 | 48 | 0.001201549 |
| Weighted A star 2nd h(n) 1st Alpha | 48 | 168 | 48 | 0.001229609 |
| Weighted A star 2nd h(n) 2nd Alpha | 48 | 168 | 48 | 0.001207908 |

| TEST CASE 3 |  |  |  |  | 
| :-: | :-: | :-: | :-: | :-: |
| Algorithm | Goal Depth | Number of Explored States | Number of Unique Explored States | Time (s)
| BFS | 68 | 9390 | 9390 | 0.143632015 |
| BFS with getHashBFS | 68 | 3329 | 3329 | 0.0436848 |
| IDS | 68 | 244197 | 7258 | 1.971885284 |
| A star 1st h(n) | 68 | 6953 | 6755 | 0.123794459 |
| A star 2nd h(n) | 68 | 9693 | 9350 | 0.153690891 |
| Weighted A star 1st h(n) 1st Alpha | 68 | 5900 | 5698 | 0.108225719 |
| Weighted A star 1st h(n) 2nd Alpha | 68 | 3899 | 3628 | 0.068949139 |
| Weighted A star 2nd h(n) 1st Alpha | 68 | 8814 | 8446 | 0.14363122 |
| Weighted A star 2nd h(n) 2nd Alpha | 68 | 844 | 645 | 0.010990302 |

| TEST CASE 4 |  |  |  |  | 
| :-: | :-: | :-: | :-: | :-: |
| Algorithm | Goal Depth | Number of Explored States | Number of Unique Explored States | Time (s)
| BFS | 92 | 1004 | 1004 | 0.012281656 |
| BFS with getHashBFS | 92 | 354 | 354 | 0.003471533 |
| IDS | 92 | 126219 | 1004 | 0.994849761 |
| A star 1st h(n) | 92 | 1013 | 1001 | 0.013907726 |
| A star 2nd h(n) | 92 | 1013 | 1001 | 0.013915038 |
| Weighted A star 1st h(n) 1st Alpha | 92 | 1003 | 987 | 0.013679162 |
| Weighted A star 1st h(n) 2nd Alpha | 92 | 912 | 846 | 0.011788822 |
| Weighted A star 2nd h(n) 1st Alpha | 92 | 1003 | 987 | 0.01364247 |
| Weighted A star 2nd h(n) 2nd Alpha | 92 | 912 | 846 | 0.011691968 |

BFS with getHashBFS is faster than normal BFS since the explored states are less.

#### Notes:

For executing the codes it's better to run code files provided in the project zip file.

There's a for loop for getting input files and running the implementations meaning that the results of all the 4 provided test cases will be shown in a single run.